In [37]:
pip install nbformat

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.offline import plot
import folium
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import re

df = pd.read_csv("csv_files/Archivochicago1limpio.csv")
df

,Unnamed: 0.1,Unnamed: 0,title,guest_favorite,rating,number_reviews,type_host,hosting_time,price,all_reviews,complete_data_list
0,0,0,Downtown|King|Rooftop|Fitness|Top Flr City View,False,4.85,NaN,Superhost,1 year hosting,177.0,"['Great stay😊', 'I had a a great experience. E...",['5 guests · 2 bedrooms · 2 beds · 1 bath']
1,1,1,BOHEMIAN LODGE #1 (clean & affordable travel),True,4.86,153.0,Superhost,8 years hosting,242.0,['Joyce is a great host and made everything to...,['1 double bed · Private attached bathroom']
2,2,2,The Colorful Little Garden Apartment,True,4.98,49.0,Superhost,1 year hosting,197.0,"['Very nice, a bit cramped for tall people, ve...",['3 guests · 1 bedroom · 1 bed · 1 bath']
3,3,3,Sonder Market Hall | Studio Apartment,False,4.70,NaN,normal_host,3 years hosting,124.0,"['Great location, easy walk to the L train and...",['2 guests · Studio · 1 bed · 1 bath']
4,4,4,Private Studio Apt in the Heart of Wicker Park!,False,4.77,NaN,normal_host,9 years hosting,235.0,['This was my first air bnb experience and it ...,['2 guests · Studio · 1 bed · 1 bath']
...,...,...,...,...,...,...,...,...,...,...,...
136,136,136,Great Location Studio II,True,4.82,522.0,Superhost,11 years hosting,119.0,['Very functional apartment for a couple (no m...,['2 guests · Studio · 1 bed · 1 bath']
137,137,137,Creative Wrigley Flat,False,NaN,NaN,Superhost,12 years hosting,55.0,[],['2 guests · 1 bedroom · 1 bed · 1 bath']
138,138,138,Charming 1BR Apartment close to Shops,False,4.72,NaN,normal_host,2 years hosting,101.0,['Great location and comfortable king bed! We ...,['2 guests · 1 bedroom · 1 bed · 1 bath']
139,139,139,G1. Next to Downtown! Best location,False,4.76,NaN,normal_host,3 years hosting,248.0,['great place\nclose to public transport\ngrea...,['1 bed · Shared bathroom']


In [39]:
# Asegurarse de que las columnas 'price' y 'rating' no tengan valores nulos
df = df.dropna(subset=['price', 'rating'])

# Filtrar números desde la columna 'all_reviews' usando regex (esto depende del formato)
df['number_reviews'] = df['number_reviews'].fillna(0).astype(float)

C:\Users\stehu\AppData\Local\Temp\ipykernel_7224\3404619568.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [40]:
df['number_reviews']

0        0.0
1      153.0
2       49.0
3        0.0
4        0.0
       ...  
135    114.0
136    522.0
138      0.0
139      0.0
140    334.0
Name: number_reviews, Length: 111, dtype: float64

In [41]:
# Relación entre precio y rating (scatter plot)
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='rating', y='price', hue='guest_favorite', palette='coolwarm', size='n_reviews', sizes=(20, 200), alpha=0.6)
plt.title('Relación entre Precio y Rating')
plt.xlabel('Rating')
plt.ylabel('Precio')
plt.show()



# Gráfico de KNN (ejemplo para predicción de precios)
X = df[['rating', 'n_reviews']].values  # Features
y = df['price'].values  # Target variable (Precio)

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo KNN
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

# Predecir
y_pred_knn = knn.predict(X_test)

# Evaluación del modelo
print(f"Error Cuadrático Medio (KNN): {mean_squared_error(y_test, y_pred_knn)}")
print(f"R² (KNN): {r2_score(y_test, y_pred_knn)}")

# Gráfico comparando predicciones y valores reales
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_knn)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=2)
plt.xlabel('Valor Real')
plt.ylabel('Valor Predicho (KNN)')
plt.title('Comparación entre Precio Real y Predicho usando KNN')
plt.show()

# Modelo Random Forest (para predicción de precios)
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predecir
y_pred_rf = rf.predict(X_test)

# Evaluación del modelo
print(f"Error Cuadrático Medio (Random Forest): {mean_squared_error(y_test, y_pred_rf)}")
print(f"R² (Random Forest): {r2_score(y_test, y_pred_rf)}")

# Gráfico comparando predicciones y valores reales (Random Forest)
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_rf)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=2)
plt.xlabel('Valor Real')
plt.ylabel('Valor Predicho (Random Forest)')
plt.title('Comparación entre Precio Real y Predicho usando Random Forest')
plt.show()

ValueError: Could not interpret value `n_reviews` for `size`. An entry with this name does not appear in `data`.

<Figure size 1000x600 with 0 Axes>

In [42]:
#grafico 3d para el streamlit
fig = px.scatter_3d(df, x='rating', y='price', z='number_reviews', 
                    color='guest_favorite', size='number_reviews', 
                    title='Relación Precio-Rating-Número de Reviews')


plot(fig)

'temp-plot.html'

: 

In [33]:
np.random.seed(42)  
df['lat'] = np.random.uniform(low=37.70, high=37.80, size=len(df))  # Latitud
df['lon'] = np.random.uniform(low=-122.50, high=-122.40, size=len(df))  # Longitud

# Crear un mapa interactivo usando Folium basado en las coordenadas
mapa = folium.Map(location=[37.7749, -122.4194], zoom_start=12)  # Centrado en San Francisco

# Añadir un marcador para cada alojamiento
for index, row in df.iterrows():
    folium.Marker([row['lat'], row['lon']], 
                  popup=f"Precio: {row['price']}, Rating: {row['rating']}").add_to(mapa)

# Mostrar el mapa
mapa
